<a href="https://colab.research.google.com/github/candelaquintero/03MIAR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_CandelaGarciaQuintero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Candela García Quintero  <br>
Url: https://https://github.com/candelaquintero/03MIAR---Algoritmos-de-Optimizacion<br>
Problema:

>2. Organizar los horarios de partidos de La Liga<br>







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
El espacio de soluciones se representa utilizando la matriz dp. Cada celda dp[i][j] de esta matriz representa la máxima audiencia posible al asignar los primeros i partidos en los primeros j horarios. Además se utiliza otra matriz horarios_asignados para almacenar los horarios asignados a cada partido en la asignación óptima.
- ¿Cual es la función objetivo? La función objetivo es maximizar la audiencia total de la jornada de partidos. Esta audiencia total se calcula sumando la audiencia de cada partido asignado en la solución óptima.
- ¿Como implemento las restricciones?
Las restricciones se han implementado en el algoritmo de programación dinámica. Las restricciones incluyen:
Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes
Cada partido debe ser asignado a un horario exactamente una vez.
No se pueden asignar más partidos de los disponibles.
Los equipos de categoria A no pueden enfrentarse a los equipos de categoria C.

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

La complejidad de este algoritmo de programación dinámica es de O(n⋅m), donde n es el número de partidos posibles y m es el número de horarios disponibles.

El algoritmo crea una matriz dp de tamaño (n+1)(m+1), donde n es el número de partidos posibles y m es el número de horarios disponibles. Por lo tanto, O(n⋅m) de tiempo y espacio.

Luego, se llena esta matriz dp utilizando dos bucles for, cada uno de los cuales tiene O(n⋅m) iteraciones. En cada iteración, se calcula la audiencia para el partido actual y se compara con la mejor audiencia. Por lo tanto, O(n⋅m).

Finalmente, después de llenar la matriz dp, se reconstruye la asignación óptima de horarios. Esto toma O(n+m) tiempo, ya que se recorren las matrices dp y
horarios_asignados.

Por lo tanto, la complejidad total del algoritmo es O(n⋅m).

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Se va a utilizar un algoritmo de programación dinámica ya que el objetivo principal de la función es maximizar el resultado.

In [1]:
import itertools

# equipos en cada categoría
equipos_a = ["A1", "A2", "A3"]
equipos_b = ["B" + str(i) for i in range(1, 12)]
equipos_c = ["C" + str(i) for i in range(1, 7)]

#coeficientes de reducción de audiencia según el horario
coeficientes_horarios = {
    ("Viernes", "20h"): 0.4,
    ("Sábado", "12h"): 0.55,
    ("Sábado", "16h"): 0.7,
    ("Sábado", "18h"): 0.8,
    ("Sábado", "20h"): 1,
    ("Domingo", "12h"): 0.45,
    ("Domingo", "16h"): 0.75,
    ("Domingo", "18h"): 0.85,
    ("Domingo", "20h"): 1,
    ("Lunes", "20h"): 0.4
}

#audiencia de los partidos según las combinaciones de equipos
audiencia_partidos = {
    ("A", "A"): 2,
    ("A", "B"): 1.3,
    ("A", "C"): 1,
    ("B", "B"): 0.9,
    ("B", "C"): 0.75,
    ("C", "C"): 0.47
}

def calcular_audiencia_partido(horario, equipos):
    audiencia_base = audiencia_partidos[(equipos[0][0], equipos[1][0])]
    coeficiente_horario = coeficientes_horarios[(horario[0], horario[1])]
    audiencia = audiencia_base * coeficiente_horario
    return audiencia

def generar_posibles_partidos():
    posibles_partidos = []
    for categoria1, categoria2 in itertools.combinations([equipos_a, equipos_b, equipos_c], 2):
        for equipo1 in categoria1:
            for equipo2 in categoria2:
                posibles_partidos.append((equipo1, equipo2))
    return posibles_partidos

def asignar_horarios_jornada(partidos, horarios):
    #se crea una matriz para almacenar los valores óptimos
    # dp[i][j] representa la máxima audiencia para asignar los primeros i partidos en los primeros j horarios
    dp = [[0] * (len(horarios) + 1) for _ in range(len(partidos) + 1)]
    horarios_asignados = [[[] for _ in range(len(horarios) + 1)] for _ in range(len(partidos) + 1)]

    for i in range(1, len(partidos) + 1):
        for j in range(1, len(horarios) + 1):
            horario_actual = horarios[j - 1]
            partido_actual = partidos[i - 1]
            audiencia_actual = calcular_audiencia_partido(horario_actual, partido_actual)

            #caso 1: no asignar el partido actual
            dp[i][j] = dp[i][j - 1]
            horarios_asignados[i][j] = horarios_asignados[i][j - 1]

            #caso 2: asignar el partido actual
            if audiencia_actual + dp[i - 1][j - 1] > dp[i][j]:
                dp[i][j] = audiencia_actual + dp[i - 1][j - 1]
                horarios_asignados[i][j] = horarios_asignados[i - 1][j - 1] + [horario_actual]

    return horarios_asignados[len(partidos)][len(horarios)]

def calcular_audiencia_total(jornada):
    audiencia_total = sum(calcular_audiencia_partido(horario, partido) for horario, partido in jornada)
    return audiencia_total

#encontramos la mejor jornada
posibles_partidos = generar_posibles_partidos()
horarios = list(coeficientes_horarios.keys())
horarios_optimos = asignar_horarios_jornada(posibles_partidos, horarios)
jornada_optima = [(horario, partido) for horario, partido in zip(horarios_optimos, posibles_partidos)]
mejor_audiencia = calcular_audiencia_total(jornada_optima)

#printeamos los resultados
print("Mejor jornada:")
for horario, partido in jornada_optima:
    print(f"{horario[0]} {horario[1]}: {partido[0]} vs {partido[1]}")
print("Audiencia total:", mejor_audiencia)

Mejor jornada:
Viernes 20h: A1 vs B1
Sábado 12h: A1 vs B2
Sábado 16h: A1 vs B3
Sábado 18h: A1 vs B4
Sábado 20h: A1 vs B5
Domingo 12h: A1 vs B6
Domingo 16h: A1 vs B7
Domingo 18h: A1 vs B8
Domingo 20h: A1 vs B9
Lunes 20h: A1 vs B10
Audiencia total: 8.97
